## run-length encode predicted masks

In [1]:
import utils
import cv2
import glob
import os

%load_ext Cython

In [2]:
%%cython
#%%cython --annotate
from cython cimport view

cimport cython
cimport numpy as np
cimport openmp
import numpy as np

@cython.boundscheck(False)
def runlen_encode(img):
    cdef np.uint8_t[::1] c_img = img.reshape(img.shape[0] * img.shape[1], order='F')
    runs = []
    cdef int npixels = len(c_img)
    cdef np.uint8_t c
    
    cdef int run_start = 1
    cdef int run_length = 0
    for i in range(npixels):
        c = c_img[i]
        if c == 0:
            if run_length != 0:
                # for kaggle, pixels are numbered from 1, hence the + 1
                runs.append((run_start + 1, run_length))
                run_length = 0
        else:
            if run_length == 0:
                run_start = i
            run_length += 1
    
    if run_length != 0:
        # for kaggle, pixels are numbered from 1, hence the + 1
        runs.append((run_start + 1, run_length))
    return runs

In [3]:
#fname = '../../preds_2016_06_05/000000.png'
fname = '../../data/train/1_1_mask.tif'
img = img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
print img.max()

runs = runlen_encode(img)

255


In [4]:
import sys

PRED_DIR='../../preds'
OUT_SUBMISSION = '../../submission.csv'

test_fnames = ['%d.png' % i for i in range(1,5509)]

with open(OUT_SUBMISSION, 'w') as f:
    f.write('img,pixels\n')
    for i, fname in enumerate(test_fnames):
        img = cv2.imread(os.path.join(PRED_DIR, fname), cv2.IMREAD_GRAYSCALE)
        if i % 10 == 0:
            print '\r%d / %d' % (i, len(test_fnames)),
            sys.stdout.flush()
            
        runs = runlen_encode(img)
        runtext = ' '.join(['%d %d' % rr for rr in runs])
        f.write('%d,%s\n' % (i + 1, runtext))

5500 / 5508
